 <p style="font-family: Arial; font-size:4.4em;color:#FFBF00;"> Optimal XPS fit </p>

# Copyrights

Analysis performed by 

Measurements performed by ... at ... 

Supervised by 

#### Note on FAIR principle:

    For the scientific transparency and verification of results obtained and commmunicated to the public after using a modified version of this work, You (as the recipient of the source code and author of this modified version, used to produce the published results in scientific communications) commit to make this modified source code available in a repository that is easily and freely accessible for a duration of five years after the communication of the obtained results.

### <font color='red'>Optimal XPS fit copyrights</font>

Copyright &copy; 2023  Antoine Roose

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>. 

Personal email: roose.devresearcher@gmail.com

Github : <a href="https://github.com/roose970">roose970</a>

Special thanks to:
- Thorsten Bartels-Rausch (Paul Scherrer Institut - Switzerland)
- Juan Florez Ospina (Paul Scherrer Institut - Switzerland)
- Markus Ammann (Paul Scherrer Institut - Switzerland)
- Luca Artiglia (Paul Scherrer Institut - Switzerland)




## Getting started

To use this notebook, it requires to know first how to handle Jupyter notebook and to have some basics knowledge of python. 

The readme file and manual are good references to know where to start

## Packages

In [1]:
#List of package 

#Base package
import pandas as pd #Handling of datafrane
import numpy as np #Mathematical function
import matplotlib.pyplot as plt #ploting the results
import matplotlib.gridspec as gridspec #Grid to plot multiple graph on the same figure, used for report figures

#packages used for XPS analysis
import nmrglue 
from nmrglue import lineshapes1d as ls #use to get nice pseudovoigt function

import lmfit #Fitting package
from lmfit.models import Model #function to create model to fit
from lmfit import minimize #fitting algorithm

#Package to handle datafiles
from igor.binarywave import load as loadibw #read Igor binary files .ibw provided by the analyzer 
import h5py #generate and handle HDF5 file

## Functions definition

file handling

In [ ]:
#Function for igor extraction
import os.path
import json
import re
import csv

from pprint import pformat

import click


def save_to_file(content, filepath, mode="x",
                 csv_headers=False, json_mini=False):
    """'Safe' interactive file saver - content should be a dict or string"""
    # Get the extension
    ext = os.path.splitext(filepath)[1]

    # Write the contents of the file, or ask for alternative filename
    try:
        with open(filepath, mode) as outfile:
            if ext == ".json":
                ind = None if json_mini else 4
                json.dump(content, outfile, indent=ind, sort_keys=True)
            elif ext in {".csv", ".tsv"}:
                _delimiter = "," if ext == ".csv" else "\t"
                csvwriter = csv.writer(outfile, delimiter=_delimiter,
                                       quotechar='"',
                                       quoting=csv.QUOTE_NONNUMERIC)
                if csv_headers:
                    csvwriter.writerow(content["labels"])
                for line in content["data"]:
                    csvwriter.writerow(line)
            else:
                outfile.write(content)
    except FileExistsError:
        click.secho("\n{} already exists!".format(filepath), fg="yellow")
        if input("Do you want to overwrite it? (y/N): ").lower() == "y":
            save_to_file(content, filepath, mode="w",
                         csv_headers=csv_headers, json_mini=json_mini)
        else:
            resp = input("Rename or cancel? (r/C): ")
            if resp == "r":
                new_filename = input("New filename ({}): ".format(ext))
                directory = os.path.dirname(filepath)
                new_filepath = os.path.join(directory, new_filename + ext)
                save_to_file(content, new_filepath, mode="x",
                             csv_headers=csv_headers, json_mini=json_mini)
            else:
                click.secho("File not saved", fg="red")


def process_notes(notes):
    """Splits a byte string into an dict"""
    # Decode to UTF-8, split at carriage-return, and strip whitespace
    note_list = list(map(str.strip, notes.decode(errors='ignore').split("\r")))
    note_dict = dict(map(fill_blanks, [p.split(":") for p in note_list]))

    # Remove the empty string key if it exists
    try:
        del note_dict[""]
    except KeyError:
        pass
    return note_dict


def fill_blanks(lst):
    """Convert a list (or tuple) to a 2 element tuple"""
    try:
        return (lst[0], from_repr(lst[1]))
    except IndexError:
        return (lst[0], "")


def from_repr(s):
    """Get an int or float from its representation as a string"""
    # Strip any outside whitespace
    s = s.strip()
    # "NaN" and "inf" can be converted to floats, but we don't want this
    # because it breaks in Mathematica!
    if s[1:].isalpha():  # [1:] removes any sign
        rep = s
    else:
        try:
            rep = int(s)
        except ValueError:
            try:
                rep = float(s)
            except ValueError:
                rep = s
    return rep


def pprint(data):
    """
    Format things into lines to get nicer printing
    Function is taken from https://github.com/wking/igor/test/test.py"""
    lines = pformat(data).splitlines()
    print('\n'.join([line.rstrip() for line in lines]))


def flatten(lst):
    """Completely flatten an arbitrarily-deep list"""
    return list(_flatten(lst))


def _flatten(lst):
    """Generator for flattening arbitrarily-deep lists"""
    for item in lst:
        if isinstance(item, (list, tuple)):
            yield from _flatten(item)
        elif item not in (None, "", b''):
            yield item

In [ ]:
# openibw : Function to read XPS files
def openibw(FilenameIgor):
    RawIgor=loadibw(FilenameIgor)
    dataIgor= RawIgor['wave']
    
    # Get the data numpy array and convert to a simple list
    wData = np.nan_to_num(dataIgor['wData']).tolist()
    
    # Get the labels and tidy them up into a list
    labels = list(map(bytes.decode,flatten(dataIgor['labels'])))

    # Get the notes and process them into a dict
    notes = process_notes(dataIgor['note'])

    #compute KE scale from data
    dataIgorheader=dataIgor['wave_header']
    Elow=dataIgorheader['sfB'][0]
    Estep=dataIgorheader['sfA'][0]
    Nstep=dataIgorheader['nDim'][0]
    Ehigh=Elow+Estep*(Nstep)
    Nint=dataIgorheader['nDim'][1]
    KE=[np.arange(Elow,Ehigh,Estep)]
    
    #assemble KE and counts
    Spectra=[0.0 for i in range(Nstep)]

    for i in range(Nstep):
        tempdat=[]
        if dataIgorheader['nDim'][2]!=0:
            for j in range(Nint):
                tempdat=tempdat+wData[i][j]
        else:
            tempdat=tempdat+wData[i]
        Spectra[i]=[np.round(KE[0][i],1)]+tempdat
    
    #Reshuffling for temporality
    Spectratemp=pd.DataFrame(Spectra).rename({0: 'KE'}, axis=1)
    dfSpectra=Spectratemp.copy()
    n=1
    for i in range(dataIgorheader['nDim'][2]):
        for j in range(dataIgorheader['nDim'][1]):
            dfSpectra[n]=Spectratemp[(j)*(dataIgorheader['nDim'][2])+(i+1)]
            n+=1
    
    return dfSpectra,notes
    

model creation

In [ ]:
# shirley_calculate : definition of the shirley background. The min and max intensity are set by hand and can be fitted

def shirley_calculate(x, y, Imin=0.0, Imax=0.0, tol=1e-5, maxit=10):
# https://github.com/kaneod/physics/blob/master/python/specs.py
 
	# Make sure we've been passed arrays and not lists.
	#x = array(x)
	#y = array(y)

	# Sanity check: Do we actually have data to process here?
	#print(any(x), any(y), (any(x) and any(y)))
	if not (any(x) and any(y)):
		print("One of the arrays x or y is empty. Returning zero background.")
		return x * 0

	# Next ensure the energy values are *decreasing* in the array,
	# if not, reverse them.
	if x[0] < x[-1]:
		is_reversed = True
		x = x[::-1]
		y = y[::-1]
	else:
		is_reversed = False

	# Locate the biggest peak.
	maxidx = abs(y - y.max()).argmin()
	
	# It's possible that maxidx will be 0 or -1. If that is the case,
	# we can't use this algorithm, we return a zero background.
	if maxidx == 0 or maxidx >= len(y) - 1:
		print("Boundaries too high for algorithm: returning a zero background.")
		return x * 0
	
	# Locate the minima either side of maxidx.
	lmidx = abs(y[0:maxidx] - y[0:maxidx].min()).argmin()
	rmidx = abs(y[maxidx:] - y[maxidx:].min()).argmin() + maxidx

	xl = x[lmidx]
	yl = Imin
	xr = x[rmidx]
	yr = Imax
	
	# Max integration index
	imax = rmidx - 1
	
	# Initial value of the background shape B. The total background S = yr + B,
	# and B is equal to (yl - yr) below lmidx and initially zero above.
	B = y * 0
	B[:lmidx] = yl - yr
	Bnew = B.copy()
	
	it = 0
	while it < maxit:
		# Calculate new k = (yl - yr) / (int_(xl)^(xr) J(x') - yr - B(x') dx')
		ksum = 0.0
		for i in range(lmidx, imax):
			ksum += (x[i] - x[i + 1]) * 0.5 * (y[i] + y[i + 1] - 2 * yr - B[i] - B[i + 1])
		k = (yl - yr) / ksum
		# Calculate new B
		for i in range(lmidx, rmidx):
			ysum = 0.0
			for j in range(i, imax):
				ysum += (x[j] - x[j + 1]) * 0.5 * (y[j] + y[j + 1] - 2 * yr - B[j] - B[j + 1])
			Bnew[i] = k * ysum
		# If Bnew is close to B, exit.
		#if norm(Bnew - B) < tol:
		B = Bnew - B
		#print(it, (B**2).sum(), tol**2)
		if (B**2).sum() < tol**2:
			B = Bnew.copy()
			break
		else:
			B = Bnew.copy()
		it += 1

	#if it >= maxit:
		#print("Max iterations exceeded before convergence.")
	if is_reversed:
		#print("Shirley BG: tol (ini = ", tol, ") , iteration (max = ", maxit, "): ", it)
		return (yr + B)[::-1]
	else:
		#print("Shirley BG: tol (ini = ", tol, ") , iteration (max = ", maxit, "): ", it)
		return yr + B


In [ ]:
# speak : definition of S orbital peak 
#Normalized pseudo voigt function created with ls.sim_pvoigt_fwhm functions scaled by the area

def speak(x,BE,FWHM,Area,GL):
    return ls.sim_pvoigt_fwhm(x, BE, FWHM, GL)*Area




In [ ]:
# ppeak : definition of P orbital peak 
#Sum of normalized pseudo voigt function created with ls.sim_pvoigt_fwhm functions scaled by the area with proportion 2/3 and 1/3

def ppeak(x,BE,FWHM,Area,GL,SOC=0.0):
    pvoigt1=ls.sim_pvoigt_fwhm(x, BE, FWHM, GL)*2/3*Area
    pvoigt2=ls.sim_pvoigt_fwhm(x, BE+SOC, FWHM, GL)*1/3*Area

    return pvoigt1+pvoigt2



In [ ]:
# dpeak : definition of D orbital peak 
#Sum of normalized pseudo voigt function created with ls.sim_pvoigt_fwhm functions scaled by the area with proportion 3/5 and 2/5

def dpeak(x,BE,FWHM,Area,GL,SOC=0.0):
    pvoigt1=ls.sim_pvoigt_fwhm(x, BE, FWHM, GL)*3/5*Area
    pvoigt2=ls.sim_pvoigt_fwhm(x, BE+SOC, FWHM, GL)*2/5*Area

    return pvoigt1+pvoigt2


In [ ]:
# liq3a1 : definition of 3a1 orbital for liquid phase water
def liq3a1(x,BE,FWHM,Area,GL,splitting=0.0):
    pvoigt1=ls.sim_pvoigt_fwhm(x, BE, FWHM, GL)*Area/2
    pvoigt2=ls.sim_pvoigt_fwhm(x, BE+splitting, FWHM, GL)*Area/2

    return pvoigt1+pvoigt2



In [ ]:
# Aspeak : definition of assymetric S orbital peak 
#http://www.casaxps.com/help_manual/line_shapes.htm
#Exponential Asymmetric Blend Based upon Voigt-type Line-Shapes (using sum pseudovoigt function)

def Aspeak(x,BE,FWHM,Area,GL,k=0.0):
    result=[ls.sim_pvoigt_fwhm(x[i], BE, FWHM, GL)*Area for i in range(len(x))]
    
    for i in range(len(x)):
        if x[i]>=BE:
            result[i]=result[i]+(1-ls.sim_pvoigt_fwhm(x[i], BE, FWHM, GL))*np.exp(-k*(x[i]-BE)/FWHM)*Area
    return result

In [ ]:
# Appeak : definition of assymetric P orbital peak 
#http://www.casaxps.com/help_manual/line_shapes.htm
#Exponential Asymmetric Blend Based upon Voigt-type Line-Shapes (using sum pseudovoigt function)

def Appeak(x,BE,FWHM,Area,GL,SOC,k=0.0):
    result=[ls.sim_pvoigt_fwhm(x[i], BE, FWHM, GL)*2/3*Area+ls.sim_pvoigt_fwhm(x[i]+SOC, BE, FWHM, GL)*1/3*Area for i in range(len(x))]
    
    for i in range(len(x)):
        if x[i]>=BE:
            result[i]=result[i]+(1-ls.sim_pvoigt_fwhm(x[i], BE, FWHM, GL))*np.exp(-k*(x[i]-BE)/FWHM)*2/3*Area+(1-ls.sim_pvoigt_fwhm(x[i]+SOC, BE, FWHM, GL))*np.exp(-k*(x[i]-BE)/FWHM)*1/3*Area
    return result

In [ ]:
# Adpeak : definition of assymetric D orbital peak 
#http://www.casaxps.com/help_manual/line_shapes.htm
#Exponential Asymmetric Blend Based upon Voigt-type Line-Shapes (using sum pseudovoigt function)

def Adpeak(x,BE,FWHM,Area,GL,SOC,k=0.0):
    result=[ls.sim_pvoigt_fwhm(x[i], BE, FWHM, GL)*3/5*Area+ls.sim_pvoigt_fwhm(x[i]+SOC, BE, FWHM, GL)*2/5*Area for i in range(len(x))]
    
    for i in range(len(x)):
        if x[i]>=BE:
            result[i]=result[i]+(1-ls.sim_pvoigt_fwhm(x[i], BE, FWHM, GL))*np.exp(-k*(x[i]-BE)/FWHM)*3/5*Area+(1-ls.sim_pvoigt_fwhm(x[i]+SOC, BE, FWHM, GL))*np.exp(-k*(x[i]-BE)/FWHM)*2/5*Area
    return result

Graphs

In [ ]:
#cell to implement a longer list of random color for graphs. colors variable is names colors
from random import randint
colors = []
for i in range(200):    
    colors.append('#%06X' % randint(0, 0xFFFFFF))

In [ ]:
# Spectrum_check : Function to plot a report figure of the raw data to determine if some spectra has to be removed

def Spectrum_check(specdatabase,removed):
    col_names=specdatabase.columns
    modifieddata=specdatabase.copy().drop(labels=removed,axis=1)
    modifiedcol_names=modifieddata.columns
    averageddata=modifieddata.iloc[0:,1:].mean(axis=1)
    stddata=modifieddata.iloc[0:,1:].std(axis=1)

    ###############################
    plt.figure(figsize=(20,18))
    G=gridspec.GridSpec(3,10)

    #Stack graph
    offset=0.0
    axes_1 = plt.subplot(G[0:2,0:3])
    for i in range(1,len(modifiedcol_names)):
        axes_1.plot(modifieddata[modifiedcol_names[0]],modifieddata[modifiedcol_names[i]],label=modifiedcol_names[i],color=colors[modifiedcol_names[i]])
    axes_1.errorbar(modifieddata[modifiedcol_names[0]],averageddata,yerr=stddata,label='Average',linewidth=4.0,color='black')
    axes_1.set_xlabel('KE (eV)')
    axes_1.set_ylabel('Counts (A.U.)')
    axes_1.legend(loc="lower center", bbox_to_anchor=(0.5, -0.55, 0, 0), ncol = 5)

    #Contour plot
    axes_2 = plt.subplot(G[:2,4:10])
    delta=np.round(round(specdatabase[col_names[0]][1],1)-round(specdatabase[col_names[0]][0],1),1)
    X=np.arange(round(specdatabase[col_names[0]][0],1),round(specdatabase[col_names[0]][specdatabase.index[-1]],1)+delta,delta)
    Y=np.arange(0,len(range(1,len(col_names)))+1)
    x,y=np.meshgrid(X,Y)
    Z=np.transpose(specdatabase.iloc[0:,1:].values.flatten().reshape(len(specdatabase.iloc[0:,1:].values),len(Y)-1)) #reshape with len(specdatabase.iloc[0:,1:].values) as first values avoid a bug but does not know why yet such a bug appears
    axes_2.pcolormesh(x+(delta/2),y+0.5,Z, vmin=np.min(Z), vmax=np.max(Z), shading='auto')
    axes_2.set_yticks(Y, minor=True)
    axes_2.grid(which='both',axis='y',linestyle='-.',alpha=0.3)
    axes_2.set_xlabel('KE (eV)')
    axes_2.set_ylabel('Spectrum')

    #Line profile
    axes_3 = plt.subplot(G[2:,4:10])
    sumval=specdatabase.iloc[0:,1:].sum(axis=0)
    axes_3.plot(np.arange(1,len(range(1,len(col_names)))+1), sumval)
    axes_3.set_xlabel('Spectrum')
    axes_3.set_ylabel('Sum counts (A.U.)')

Useful function that would need some user's hand modification

In [ ]:
# Function to allow the fitting of reference spectra into another. The reference spectra is transformed into a model with this function.
# The user has to modified the function name and the dataset variables to implement the reference spectra he has previously uploaded
 
def set_your_funcion_name(x,y,shift=0.0,scale=1):
    #the function take to parameter to be fit, 'shift' to take into account any possible shift in kinetic energy, 'scale' to allow the scaling of intensity.
    #parameter x and y are data from the spectra to fit (independent variables)

    global set_your_dataframe_name #here write the name of the dataframe that contain the average spectra

    ydata=y.copy()
    
    for i in range(len(ydata)):
        if (set_your_dataframe_name.loc[set_your_dataframe_name['BE']==(np.round(x[i]+shift,1))].empty==False):
            ydata[i]=set_your_dataframe_name['Counts'].loc[set_your_dataframe_name['BE']==(np.round(x[i]+shift,1))].values
            ydata[i]=ydata[i]*scale
        

    return ydata

#to call the function as a model in the cell to create nodel the user has to use the following commands:
# peakname=Model(set_your_function_name,independent_vars=['x','y'],prefix='peakname_')
# and set/modify the parameters

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!START OF THE ANALYSIS HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

## Title

### subtitle

Comment

In [ ]:
#Open file, read and store data/metadata

################################################################################################################
dat1,info1=openibw(r"yourpathway\yourfilename.ibw") #chage filename and variable name according to what is needed
#the variable name should be consistent then with the following cells
dat1_hv=Your_photon_energy #change the photon energy
################################################################################################################

#comment/uncomment next section as you need
#print(dat1) #print data
#print(info1) #print metadata

In [ ]:
#print all spectra to check that there is no spectrum outstanding

dat1_removed=[] #add here the numbers of the spectra you want to remove to take a full range # [i for i in range(low,high)] careful the first speactrum index is 1
Spectrum_check(dat1,dat1_removed)

In [ ]:
#Average and standard deviation computation

tempdata=dat1.copy().drop(labels=dat1_removed,axis=1)
tempRawspectra=tempdata.iloc[0:,1:].mean(axis=1)
tempstdRawspectra=tempdata.iloc[0:,1:].std(axis=1)

tempBE=dat1_hv-dat1['KE']


Rawspectra=pd.concat([tempBE,tempRawspectra,tempstdRawspectra],axis=1)
Rawspectra.columns=['BE','Counts','std']

#plot spectra
graph,axis=plt.subplots(1)
axis.plot(Rawspectra['BE'],Rawspectra['Counts'])
axis.invert_xaxis()

In [ ]:
################################################################
UpperBE=the_upper_BE
LowerBE=the_lower_BE
################################################################

#change variable name of rawspectra
raw1=Rawspectra.loc[Rawspectra['BE']<=UpperBE].copy()
raw2=raw1.loc[raw1['BE']>=LowerBE].copy()


#Change variable name Ibeg and Iend
################################################################
Ibeg=raw2.iloc[0].name
Iend=raw2.iloc[-1].name
################################################################

#print(Ibeg)
#print(Iend)

#uncomment if you want to plot the average spectra again
#plot the graph
# graph,axis=plt.subplots(1)
# axis.plot(Rawspectra['BE'],Rawspectra['Counts'])
# axis.invert_xaxis()

Start fitting

In [ ]:
#Here the biggest job, set the peaks/background for each spectra
#####################################################################################
#####################################################################################

#All variable name should be different for each pekas/background and consistent with their prefix (for the user to know what belongs to each peaks
# )
#Spectrum 1
xdatgs1=Rawspectra['BE'][Ibeg:Iend].values  #here set the x values of the spectrum to fit
ydatgs1=Rawspectra['Counts'][Ibeg:Iend].values #here set the y value of the spectrum to fit

#definition of background
#All XPS spectra should get a shirley or a tougaard (not implemented yet) background. And some recent studies shows that fitting the backgrounds with the peaks is more accurate
backgs1=Model(shirley_calculate,independent_vars=['x','y','tol','maxit'],prefix='backgs1_') #model for the shirley background
pars=backgs1.make_params() #creation of the parameter (should be done only for the first one then use pars.update(your_peakvariable.make_params()) )
#once the parameters variable is created, you should assign initial values, and eventually boundary conditions or fixed the variable (with the option vary=False)
#to assign values use the command pars['prefix+name_of_the_parameter'].set(value=initialvalue[,min=minvalue,max=maxvalue,vary=False])
pars['backgs1_Imin'].set(value=20.0,min=-450.0,max=250) #set the parameter / high binding energy
pars['backgs1_Imax'].set(value=0.0,min=-450.0,max=150) #set the parameter / low binding energy

#then define your peaks. Below an example of a gas phase VB of water
#definition of peaks
#peak 1 1b1
pgs11=Model(speak,prefix='pgs11_') #model for the dpeak
pars.update(pgs11.make_params()) #update the parameter(important to not create a new set of parameter)
pars['pgs11_BE'].set(value=13,min=12,max=14) #set the parameter
pars['pgs11_FWHM'].set(value=0.4,min=0.0,max=4.0) #set the parameter
pars['pgs11_Area'].set(value=5000,min=0.0) #set the parameter
pars['pgs11_GL'].set(value=0.001,min=0.0,max=1.0,vary=False) #set the parameter

#peak 2 3a1
pgs12=Model(speak,prefix='pgs12_') #model for the dpeak
pars.update(pgs12.make_params()) #update the parameter(important to not create a new set of parameter)
pars['pgs12_BE'].set(value=15.3,min=14,max=17) #set the parameter
pars['pgs12_FWHM'].set(value=1.15,min=0.0,max=4.0) #set the parameter
pars['pgs12_Area'].set(value=2500,min=0.0) #set the parameter
pars['pgs12_GL'].set(value=0.001,min=0.0,max=1.0,vary=False) #set the parameter

#peak 3 1b2
pgs13=Model(speak,prefix='pgs13_') #model for the dpeak
pars.update(pgs13.make_params()) #update the parameter(important to not create a new set of parameter)
pars['pgs13_BE'].set(value=19,min=18,max=21) #set the parameter
pars['pgs13_FWHM'].set(value=1.6,min=0.0,max=4.0) #set the parameter
pars['pgs13_Area'].set(value=1300,min=0.0) #set the parameter
pars['pgs13_GL'].set(value=0.001,min=0.0,max=1.0,vary=False) #set the parameter

#create model for spectrum, this model is a sum of the backgrounds and the peaks
modgs1 = backgs1 + pgs11 + pgs12 + pgs13 #create the sum model for the spectrum


#####################################################################################

#If you want to fit multiple spectra with constrained parameters add as many section to create model as there is spectra to fit.

#####################################################################################
#####################################################################################

#end of the edition, fitting is close but double check before going to the next cell
#debugging this part can be quite difficult the more spectra to fit there is.
#Check carefully that the list of parameter (pars in this example) is correctly set and has not been erased by another make_params() command
#check that you have correctly set the name of the variables in the cell, otherwise you may have some surprise in the fitting or some code error

In [ ]:
#Plot the data and initial fit to check
#The initial fit should be close to the spectra for faster fitting and better results

plt.figure(figsize=(6,12))
###############################################################################################
plt.plot(xdatgs1, ydatgs1) #in these three lines, change the variable name to the name you have set
plt.plot(xdatgs1, modgs1.eval(pars,x=xdatgs1,y=ydatgs1, shift=0.0, tol=1e-5, maxit=30), '--m', label='init_fit') #plot initial evaluation of the model created in the cell earlier. This evaluation is based on initial parameter 

#in case there is more than one spectrum to fit, an offset is added based on the maxvalue of the previous spectrum
#offset=np.max(ydatgs1) #offset the next spectra to not overlap
# ###############################################################################################

#then create as many section as required by adding the new max value to the offset.

# ###############################################################################################
# plt.plot(xdatgs2, offset+ydatgs2)
# plt.plot(xdatgs2, offset+modgs2.eval(pars,x=xdatgs2,y=ydatgs2,shift=0.0, tol=1e-5, maxit=30), '--m')

# offset=offset+np.max(ydatgs2) #offset the next spectra to not overlap note that the line is different from the previous offset
# ###############################################################################################

# #the previous section has to be copy as many time as needed. Don't forget to change the variable names


plt.legend()




In [ ]:
#Creation of the fitting function

#note: lmfit needs a 1D array to work. This function merge all the data, model and residual in one array for the fitting
# xaxis values need to be put in one dataset as well

data=[] #list initialisation

#extend the data variable as many time as there is spectrum
##############################################################################################################
data.extend(ydatgs1) #creation of the dataset for the fitting function . Change variable names as set previsouly
# data.extend(ydatgs2)
# data.extend(ydatgs3)
# data.extend(ydatgs4)
# data.extend(ydatgs5)
# data.extend(ydatgs6)
# data.extend(ydatgs7)
##############################################################################################################

data=np.array(data) #convert list to array

xaxis=[]

#extend the xaxis variable as many time as there is spectrum
#################################################################################################################
xaxis.append(xdatgs1) #fitting function needs x values as argument for the fitting procedure. change variable names
# xaxis.append(xdatgs2)
# xaxis.append(xdatgs3)
# xaxis.append(xdatgs4)
# xaxis.append(xdatgs5)
# xaxis.append(xdatgs6)
# xaxis.append(xdatgs7)
#################################################################################################################

xaxis=np.array(xaxis)

#create the objective of the fit. the fitting algorithm wants an objective that it will try to set to 0, here the residual
def objective(pars, xaxis, data):
    """Calculate total data residual for fits of models to several data sets."""
    
    res = 0.0*data[:]

    mod=[]
    
    #extend the mod variable as many time as there is spectrum
    #######################################################################################################################################
    mod.extend(modgs1.eval(pars,x=xdatgs1,y=ydatgs1, tol=1e-5, maxit=30)) #initialisation of the model. Change variable names as set previsouly
    # mod.extend(modgs2.eval(pars,x=xdatgs2,y=ydatgs2, tol=1e-5, maxit=30))
    # mod.extend(modgs3.eval(pars,x=xdatgs3,y=ydatgs3, tol=1e-5, maxit=30))
    # mod.extend(modgs4.eval(pars,x=xdatgs4,y=ydatgs4, tol=1e-5, maxit=30))
    # mod.extend(modgs5.eval(pars,x=xdatgs5,y=ydatgs5, tol=1e-5, maxit=30))
    # mod.extend(modgs6.eval(pars,x=xdatgs6,y=ydatgs6, tol=1e-5, maxit=30))
    # mod.extend(modgs7.eval(pars,x=xdatgs7,y=ydatgs7, tol=1e-5, maxit=30))
    #######################################################################################################################################

    mod=np.array(mod)
    
    res = data - mod

    return res 




In [ ]:
# Fitting, can be long, go take a moka/tea and read papers :) 

#change out variable name to not erase answer of a previous fitting in the notebook
#decrease ftol and xtol to improve the fitting up to the moment the time is not reasonable enymore (more than an hour is long)
#future development will be done to implement likelyhood based methodology (Markov chain monte carlo)
outgs1 = minimize(objective, pars, args=(xaxis,data),ftol=1e-5,xtol=1e-5)
print(outgs1.message) #message fit succeded or not... 

In [ ]:
#print the report
lmfit.report_fit(outgs1) #change the variable name accordint to the previous cell

In [ ]:
#plot a figure to resume the fit results

#################################################################################################
#according to the number of spectra you may want to edit this section. I may write a short manual
#for this purpose. For the moment internet is your friend :)
#gridspec and matplotlib webpages are a good start

#On this figures I plot on the left the spectra with their initial fit and best fit.
#Then, a plot that shows the residuals
#and finally one plot per spectrum with all the individual contribution of each peaks (up to four plots then create another cells if there is more spectra)

#to change the color, you can refer to the color code at the end of this notebook

plt.figure(figsize=(24,12)) #by editing the two number the figure size can be changed
G=gridspec.GridSpec(2,10,wspace=0.5,hspace=0.24) #create a virtual grid in the figure

#The grid created got 2 vertical section and 10 horizontal ones
#################################################################################################


#plot initial fit and best fit

axes_1 = plt.subplot(G[:,0:2]) #plot on both vertical sections and the first two horizontal sections

#######################################################################################################
#change variable name in this section
axes_1.plot(xdatgs1, modgs1.eval(pars,x=xdatgs1,y=ydatgs1, tol=1e-5, maxit=30), '--m', label='initial fit')
axes_1.plot(xdatgs1, modgs1.eval(outgs1.params,x=xdatgs1,y=ydatgs1, tol=1e-5, maxit=30), '-k', label='best fit')
axes_1.plot(xdatgs1, ydatgs1, label='VB gas phase 250eV') #edit label if needed

#an offset is created for the next spectrum 
offset=np.max(ydatgs1) #offset for the next spectrum
#######################################################################################################

#copy paste the next session as needed
#######################################################################################################
# axes_1.plot(xdatgs2, offset+modgs2.eval(pars,x=xdatgs2,y=ydatgs2, tol=1e-5, maxit=30), '--m')
# axes_1.plot(xdatgs2, offset+modgs2.eval(outgs1.params,x=xdatgs2,y=ydatgs2, tol=1e-5, maxit=30), '-k')
# axes_1.plot(xdatgs2, offset+ydatgs2, label='VB gas phase 310eV')

#the offset is updated for the next spectrum 
# offset=offset+np.max(ydatgs2) #offset for the next spectrum
#######################################################################################################


axes_1.set_xlabel('BE (eV)')
axes_1.set_ylabel('Counts (a.u.)')
axes_1.legend()
axes_1.invert_xaxis() #invert xaxis. standard for BE plot

#plot residuals
offset=0.0 #reset offset
axes_2 = plt.subplot(G[:,2:4],sharey=axes_1) #plot on both vertical sections and the third and fourth horizontal sections #yaxis is the same as the first plot

###########################################################################################################################
#change variable name in this section. Edit label if needed
axes_2.plot(xdatgs1, ydatgs1-modgs1.eval(outgs1.params,x=xdatgs1,y=ydatgs1, tol=1e-5, maxit=30), '--', label='residual VB gas phase 250eV')
axes_2.plot(xdatgs1, ydatgs1-ydatgs1,'b') #just the zero line for a better representation

#an offset is created for the next spectrum 
offset=np.max(ydatgs1) #offset for the next spectrum
###########################################################################################################################

#copy paste the next session as needed
###########################################################################################################################
#change variable name in this section. Edit label if needed
# axes_2.plot(xdatgs2, offset+ydatgs2-modgs2.eval(outgs1.params,x=xdatgs2,y=ydatgs2, tol=1e-5, maxit=30), '--', label='residual VB gas phase 310eV')
# axes_2.plot(xdatgs1, offset+ydatgs1-ydatgs1,color='orange') #just the zero line with the offset for a better representation

#the offset is updated for the next spectrum 
# offset=offset+np.max(ydatgs2) #offset for the next spectrum
###########################################################################################################################

axes_2.set_xlabel('BE (eV)')
axes_2.set_ylabel('Counts (a.u.)')
axes_2.legend()
axes_2.invert_xaxis()

#plot individual spectrum fitting

#DP1
axes_3 = plt.subplot(G[0,4:7]) #plot on first vertical section and the fith to seventh horizontal sections 
##############################################################################################################################
#change variable name in this section. Edit label if needed
axes_3.plot(xdatgs1, ydatgs1, 'b',label='VB gas phase 250eV')
background=backgs1.eval(outgs1.params,x=xdatgs1,y=ydatgs1, tol=1e-5, maxit=30) #get the value of the background
#plot each peaks. The background is added for a better representation. Suggestion: peaks should be plotted efore that that they appears first in the legend of the graph
axes_3.plot(xdatgs1, background+pgs11.eval(outgs1.params,x=xdatgs1,y=ydatgs1, tol=1e-5, maxit=30), '--',color='dimgrey', label='1b1')
axes_3.plot(xdatgs1, background+pgs12.eval(outgs1.params,x=xdatgs1,y=ydatgs1, tol=1e-5, maxit=30), '--k', label='3a1')
axes_3.plot(xdatgs1, background+pgs13.eval(outgs1.params,x=xdatgs1,y=ydatgs1, tol=1e-5, maxit=30), '--r', label='1b2')
axes_3.plot(xdatgs1, background+pgs14.eval(outgs1.params,x=xdatgs1,y=ydatgs1, tol=1e-5, maxit=30), '--g', label='O2s')
#plot the background
axes_3.plot(xdatgs1, background, '--m', label='Shirley Background')
##############################################################################################################################

axes_3.set_xlabel('BE (eV)')
axes_3.set_ylabel('Counts (a.u.)')
axes_3.legend()
axes_3.invert_xaxis()

#in case there is more spectrum here is an example of the code to plot up to four spectra
#DP2
# axes_4 = plt.subplot(G[0,7:]) #plot on first vertical section and the eighth to tenth horizontal sections 
# ##############################################################################################################################
# #change variable name in this section. Edit label if needed
# axes_4.plot(xdatgs2, ydatgs2,color='orange', label='VB gas phase 310eV')
# background=backgs2.eval(outgs1.params,x=xdatgs2,y=ydatgs2, tol=1e-5, maxit=30)
# axes_4.plot(xdatgs2, background+pgs21.eval(outgs1.params,x=xdatgs2,y=ydatgs2, tol=1e-5, maxit=30), '--',color='dimgrey', label='1b1')
# axes_4.plot(xdatgs2, background+pgs22.eval(outgs1.params,x=xdatgs2,y=ydatgs2, tol=1e-5, maxit=30), '--k', label='3a1')
# axes_4.plot(xdatgs2, background+pgs23.eval(outgs1.params,x=xdatgs2,y=ydatgs2, tol=1e-5, maxit=30), '--r', label='1b2')
# axes_4.plot(xdatgs2, background, '--m', label='Shirley Background')
# ##############################################################################################################################

# axes_4.set_xlabel('BE (eV)')
# axes_4.set_ylabel('Counts (a.u.)')
# axes_4.legend()
# axes_4.invert_xaxis()

#DP3
# axes_5 = plt.subplot(G[1,4:7]) #plot on second vertical section and the fith to seventh horizontal sections
# ##############################################################################################################################
# #change variable name in this section. Edit label if needed
# axes_5.plot(xdatgs3, ydatgs3,'g', label='VB gas phase 410eV')
# background=backgs3.eval(outgs1.params,x=xdatgs3,y=ydatgs3, tol=1e-5, maxit=30)
# axes_5.plot(xdatgs3, background+pgs31.eval(outgs1.params,x=xdatgs3,y=ydatgs3, tol=1e-5, maxit=30), '--',color='dimgrey', label='1b1')
# axes_5.plot(xdatgs3, background+pgs32.eval(outgs1.params,x=xdatgs3,y=ydatgs3, tol=1e-5, maxit=30), '--k', label='3a1')
# axes_5.plot(xdatgs3, background+pgs33.eval(outgs1.params,x=xdatgs3,y=ydatgs3, tol=1e-5, maxit=30), '--r', label='1b2')
# axes_5.plot(xdatgs3, background, '--m', label='Shirley Background')
# ##############################################################################################################################

# axes_5.set_xlabel('BE (eV)')
# axes_5.set_ylabel('Counts (a.u.)')
# axes_5.legend()
# axes_5.invert_xaxis()

#DP4
# axes_6 = plt.subplot(G[1,7:]) #plot on second vertical section and the eighth to tenth horizontal sections
# ##############################################################################################################################
# #change variable name in this section. Edit label if needed
# axes_6.plot(xdatgs4, ydatgs4,'r', label='VB gas phase 510eV')
# background=backgs4.eval(outgs1.params,x=xdatgs4,y=ydatgs4, tol=1e-5, maxit=30)
# axes_6.plot(xdatgs4, background+pgs41.eval(outgs1.params,x=xdatgs4,y=ydatgs4, tol=1e-5, maxit=30), '--',color='dimgrey', label='1b1')
# axes_6.plot(xdatgs4, background+pgs42.eval(outgs1.params,x=xdatgs4,y=ydatgs4, tol=1e-5, maxit=30), '--k', label='3a1')
# axes_6.plot(xdatgs4, background+pgs43.eval(outgs1.params,x=xdatgs4,y=ydatgs4, tol=1e-5, maxit=30), '--r', label='1b2')
# axes_6.plot(xdatgs4, background, '--m', label='Shirley Background')
# ##############################################################################################################################

# axes_6.set_xlabel('BE (eV)')
# axes_6.set_ylabel('Counts (a.u.)')
# axes_6.legend()
# axes_6.invert_xaxis()


# Saving data analysis

In [ ]:
fileName='OptimalXPSfit.ipynb' #write filename of the jupyter notebook

In [ ]:
#convert the jupyter notebook in HTML, save it before activating the cell
from nbconvert import HTMLExporter
import codecs
import nbformat
exporter = HTMLExporter()
output_notebook = nbformat.read(fileName, as_version=4)
output, resources = exporter.from_notebook_node(output_notebook)
codecs.open(fileName + '.html', 'w', encoding='utf-8').write(output)


HDF5 file (under development)

In [ ]:
my_file=h5py.File('./test_hdf5.hdf5','a')
my_group_01 = my_file.create_group('metadata')
my_group_02 = my_file.create_group('/metadata/spectra')
group_01 = my_file['/metadata']
my_dataset = group_01.create_dataset(name='demo_metadataset', data=info1,dtype=)
group_02 = my_file['/metadata/spectra']
my_dataset2 = group_01.create_dataset(name='demo_dataset', data=dat1)
my_file.close()

# Annexes

Color code matplotlib

In [ ]:
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


def plot_colortable(colors, sort_colors=True, emptycols=0):

    cell_width = 212
    cell_height = 22
    swatch_width = 48
    margin = 12

    # Sort colors by hue, saturation, value and name.
    if sort_colors is True:
        by_hsv = sorted((tuple(mcolors.rgb_to_hsv(mcolors.to_rgb(color))),
                         name)
                        for name, color in colors.items())
        names = [name for hsv, name in by_hsv]
    else:
        names = list(colors)

    n = len(names)
    ncols = 4 - emptycols
    nrows = n // ncols + int(n % ncols > 0)

    width = cell_width * 4 + 2 * margin
    height = cell_height * nrows + 2 * margin
    dpi = 72

    fig, ax = plt.subplots(figsize=(width / dpi, height / dpi), dpi=dpi)
    fig.subplots_adjust(margin/width, margin/height,
                        (width-margin)/width, (height-margin)/height)
    ax.set_xlim(0, cell_width * 4)
    ax.set_ylim(cell_height * (nrows-0.5), -cell_height/2.)
    ax.yaxis.set_visible(False)
    ax.xaxis.set_visible(False)
    ax.set_axis_off()

    for i, name in enumerate(names):
        row = i % nrows
        col = i // nrows
        y = row * cell_height

        swatch_start_x = cell_width * col
        text_pos_x = cell_width * col + swatch_width + 7

        ax.text(text_pos_x, y, name, fontsize=14,
                horizontalalignment='left',
                verticalalignment='center')

        ax.add_patch(
            Rectangle(xy=(swatch_start_x, y-9), width=swatch_width,
                      height=18, facecolor=colors[name], edgecolor='0.7')
        )

    return fig

In [ ]:
plot_colortable(mcolors.BASE_COLORS, sort_colors=False, emptycols=1)
plot_colortable(mcolors.TABLEAU_COLORS, sort_colors=False, emptycols=2)
plot_colortable(mcolors.CSS4_COLORS)
plt.show()

# End of the notebook